In [43]:
import pandas as pd
import pulp 

In [44]:
stock_df = pd.read_csv('stocks.csv')
requires_df = pd.read_csv('requires.csv')
gains_df = pd.read_csv('gains.csv')

In [45]:
stock_df.head()

,m,stock
0,m1,35
1,m2,22
2,m3,27


In [46]:
requires_df.head()

,p,m,require
0,p1,m1,2
1,p1,m2,0
2,p1,m3,1
3,p2,m1,3
4,p2,m2,2


In [47]:
gains_df.head()

,p,gain
0,p1,3
1,p2,4
2,p3,4
3,p4,5


In [48]:
#原材料の添字集合
M = list(stock_df['m'])
#製品の添字集合
P = list(gains_df['p'])

In [50]:
stock = {row.m:row.stock for row in stock_df.itertuples()}

In [51]:
stock

{'m1': 35, 'm2': 22, 'm3': 27}

In [52]:
require = {(row.p, row.m) : row.require for row in requires_df.itertuples()}

In [53]:
require

{('p1', 'm1'): 2,
 ('p1', 'm2'): 0,
 ('p1', 'm3'): 1,
 ('p2', 'm1'): 3,
 ('p2', 'm2'): 2,
 ('p2', 'm3'): 0,
 ('p3', 'm1'): 0,
 ('p3', 'm2'): 2,
 ('p3', 'm3'): 2,
 ('p4', 'm1'): 2,
 ('p4', 'm2'): 2,
 ('p4', 'm3'): 2}

In [54]:
gain = {row.p : row.gain for row in gains_df.itertuples()}

In [56]:
problem = pulp.LpProblem('LP', pulp.LpMaximize)
#決定変数は生産量 P_i
# LpVariableのdictionaryを作成
x = pulp.LpVariable.dicts('x', P, cat='Continuous')
for p in P:
    problem += x[p] >= 0
for m in M:
    problem += pulp.lpSum([require[p,m] * x[p] for p in P]) <= stock[m]
problem += pulp.lpSum([gain[p] * x[p] for p in P])    
status = problem.solve()
print("Status : ", pulp.LpStatus[status])
#print(pulp.LpStatus)
for p in P:
    print(f"Optimal sol x{p} : ", x[p].value())
print(f"Optimal value : {problem.objective.value()}")    

Welcome to the CBC MILP Solver 
Version: 2.9.0 
Build Date: Feb 12 2015 

command line - /opt/anaconda3/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/kc/5vks2chj5jg_zq9fvglb01jr0000gp/T/8678d19ddf3e411ebd2311835658ae43-pulp.mps max ratio None allow None threads None presolve on strong None gomory on knapsack on probing on branch printingOptions all solution /var/folders/kc/5vks2chj5jg_zq9fvglb01jr0000gp/T/8678d19ddf3e411ebd2311835658ae43-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 12 COLUMNS
At line 30 RHS
At line 38 BOUNDS
At line 43 ENDATA
Problem MODEL has 7 rows, 4 columns and 13 elements
Coin0008I MODEL read with 0 errors
String of None is illegal for double parameter ratioGap value remains 0
String of None is illegal for double parameter allowableGap value remains 0
String of None is illegal for integer parameter threads value remains 0
String of None is illegal for integer parameter strongBranching value rem

In [94]:
problem = pulp.LpProblem('LP', pulp.LpMaximize)
#決定変数は生産量 P_i
x = pulp.LpVariable.dicts('x', P, cat='Integer')
for p in P:
    problem += x[p] >= 0
for m in M:
    problem += pulp.lpSum([require[p,m] * x[p] for p in P]) <= stock[m]
problem += pulp.lpSum([gain[p] * x[p] for p in P])    
status = problem.solve()
print("Status : ", pulp.LpStatus[status])
print(pulp.LpStatus)
for p in P:
    print(f"Optimal sol x{p} : ", x[p].value())
print(f"Optimal value : {problem.objective.value()}")    

Status :  Optimal
{0: 'Not Solved', 1: 'Optimal', -1: 'Infeasible', -2: 'Unbounded', -3: 'Undefined'}
Optimal sol xp1 :  13.0
Optimal sol xp2 :  3.0
Optimal sol xp3 :  7.0
Optimal sol xp4 :  -0.0
Optimal value : 79.0
